<a href="https://colab.research.google.com/github/erwanplantec/FlowLenia/blob/main/FlowLenia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! nvidia-smi

Tue Oct  4 09:09:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Imports

In [ ]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

In [ ]:
try :
  import jax
  import jax.numpy as jnp
  import jax.scipy as jsp
except ModuleNotFoundError :
  print('installing jax')
  install("jax"); install("jaxlib")
  import jax
  import jax.numpy as jnp
  import jax.scipy as jsp

try :   
  from tqdm import tqdm
except ModuleNotFoundError :
  print('installing tqdm')
  install("tqdm")
  from tqdm import tqdm

import numpy as np
import random
import matplotlib.pyplot as plt
import gc
from functools import partial
import typing as t

# Utils

In [ ]:
import os
os.environ['FFMPEG_BINARY'] = 'ffmpeg'
import moviepy.editor as mvp
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
from IPython.display import HTML, display, clear_output

class VideoWriter:
  def __init__(self, filename, fps=30.0, **kw):
    self.writer = None
    self.params = dict(filename=filename, fps=fps, **kw)

  def add(self, img):
    img = np.asarray(img)
    if self.writer is None:
      h, w = img.shape[:2]
      self.writer = FFMPEG_VideoWriter(size=(w, h), **self.params)
    if img.dtype in [np.float32, np.float64]:
      img = np.uint8(img.clip(0, 1)*255)
    if len(img.shape) == 2:
      img = np.repeat(img[..., None], 3, -1)
    self.writer.write_frame(img)

  def close(self):
    if self.writer:
      self.writer.close()

  def __enter__(self):
    return self

  def __exit__(self, *kw):
    self.close()

  def show(self, **kw):
      self.close()
      fn = self.params['filename']
      display(mvp.ipython_display(fn, **kw))

In [ ]:
kx = jnp.array([
                [1., 0., -1.],
                [2., 0., -2.],
                [1., 0., -1.]
])
ky = jnp.transpose(kx)
def sobel_x(A):
  """
  A : (x, y, c)
  ret : (x, y, c)
  """
  return jnp.dstack([jsp.signal.convolve2d(A[:, :, c], kx, mode = 'same') 
                    for c in range(A.shape[-1])])
def sobel_y(A):
  return jnp.dstack([jsp.signal.convolve2d(A[:, :, c], ky, mode = 'same') 
                    for c in range(A.shape[-1])])
  
@jax.jit
def sobel(A):
  return jnp.concatenate((sobel_y(A)[:, :, None, :], sobel_x(A)[:, :, None, :]),
                         axis = 2)

In [ ]:
def center_of_mass(im):
  im = im[:, :, 0]
  mass = im.sum()
  SX, SY = im.shape[0], im.shape[1]
  x, y = jnp.arange(SX), jnp.arange(SY)
  xx, yy = jnp.meshgrid(x, y)
  X, Y = xx - SX / 2, yy - SY / 2

  # Centroids
  cx = (X * im).sum() / (mass + 1e-10)
  cy = (Y * im).sum() / (mass + 1e-10)

  z = jnp.zeros(2)
  z = z.at[0].set(cx/SX)
  z = z.at[1].set(cy/SY)

  return z

def mass(X):
  return X.sum()

In [ ]:
def conn_from_matrix(mat):
  C = mat.shape[0]
  c0 = []
  c1 = [[] for _ in range(C)]
  i = 0
  for s in range(C):
    for t in range(C):
      n = mat[s, t]
      if n:
        c0 = c0 + [s]*n
        c1[t] = c1[t] + list(range(i, i+n))
      i+=n
  return c0, c1


def conn_from_lists(c0, c1, C):
  return c0, [[i == c1[i] for i in range(len(c0))] for c in range(C)]

In [ ]:
def rollout(step_fn, c_params, A, steps, verbose = False):
  obs = np.zeros((steps+1, *A0.shape))
  obs[0] = A
  rnge = tqdm(range(steps)) if verbose else range(steps)
  for t in rnge :
    A = step_fn(A, c_params)
    obs[t+1] = A
  return obs

def rollout_final(step_fn, c_params, A, steps, verbose = False):
  return jax.lax.fori_loop(0, steps, lambda i, s : step_fn(A, c_params), A)

# Rule Space

In [ ]:
class Rule_space :
  #-----------------------------------------------------------------------------
  def __init__(self, nb_k, init_shape = (40, 40)):
    self.nb_k = nb_k
    self.init_shape = init_shape
    self.kernel_keys = 'r b w a m s h'.split()
    self.spaces = {
        "r" : {'low' : .2, 'high' : 1., 'mut_std' : .2, 'shape' : None},
        "b" : {'low' : .001, 'high' : 1., 'mut_std' : .2, 'shape' : (3,)},
        "w" : {'low' : .01, 'high' : .5, 'mut_std' : .2, 'shape' : (3,)},
        "a" : {'low' : .0, 'high' : 1., 'mut_std' : .2, 'shape' : (3,)},
        "m" : {'low' : .05, 'high' : .5, 'mut_std' : .2, 'shape' : None},
        "s" : {'low' : .001, 'high' : .18, 'mut_std' : .01, 'shape' : None},
        "h" : {'low' : .01, 'high' : 1., 'mut_std' : .2, 'shape' : None},
        'T' : {'low' : 10., 'high' : 50., 'mut_std' : .1, 'shape' : None},
        'R' : {'low' : 2., 'high' : 25., 'mut_std' : .2, 'shape' : None},
        'init' : {'low' : 0., 'high' : 1., 'mut_std' : .2, 'shape' : self.init_shape}
    }
  #-----------------------------------------------------------------------------
  def sample(self):
    kernels = {}
    for k in 'rmsh':
      kernels[k] = np.random.uniform(
          self.spaces[k]['low'], self.spaces[k]['high'], self.nb_k
      )
    for k in "awb":
      kernels[k] = np.random.uniform(
          self.spaces[k]['low'], self.spaces[k]['high'], (self.nb_k, 3)
      )
    return {
        'kernels' : kernels, 
        'T' : np.random.uniform(self.spaces['T']['low'], self.spaces['T']['high']),
        'R' : np.random.uniform(self.spaces['R']['low'], self.spaces['R']['high']),
        'init' : np.random.rand(*self.init_shape) 
    }
  #-----------------------------------------------------------------------------
  def mutate(self, params, seeds = None, mask = None):
    if seeds is None : seeds = self.make_seeds()
    mut = {'kernels' : {}}
    if mask is None :
      mask = jnp.zeros(self.nb_k)
    for i, k in enumerate("T R".split()):
      mut[k] = params[k] + jax.random.normal(
          jax.random.PRNGKey(seeds[- i - 1])
      ) * self.spaces[k]['mut_std']
    for i, k in enumerate(self.kernel_keys):
      _mask = mask if self.spaces[k]['shape'] is None else mask[:, None]
      mut['kernels'][k] = params['kernels'][k] + jax.random.normal(
          jax.random.PRNGKey(seeds[i]),
          params['kernels'][k].shape) * self.spaces[k]['mut_std'] * (1 - _mask)
    mut['init'] = params['init']
    return self.clip(mut)
  #-----------------------------------------------------------------------------
  def sample_batched(self, n):
    """return vec batched parameters set with batch size n"""
    kernels = {}
    for k in 'rmsh':
      kernels[k] = np.random.uniform(
          self.spaces[k]['low'], self.spaces[k]['high'], (n, self.nb_k)
      )
    for k in "awb":
      kernels[k] = np.random.uniform(
          self.spaces[k]['low'], self.spaces[k]['high'], (n, self.nb_k, 3)
      )
    return {
        'kernels' : kernels, 
        'T' : np.random.uniform(
            self.spaces['T']['low'], 
            self.spaces['T']['high'],
            n),
        'R' : np.random.uniform(
            self.spaces['R']['low'], 
            self.spaces['R']['high'],
            n),
        'init' : jnp.stack([np.random.rand(*self.init_shape) for _ in range(n)])
    }
  #-----------------------------------------------------------------------------
  def mut_batched(self, params, n, seeds = None, method = "classic"):
    """Return n batched parameters sets where each param set is mutated params (params + gaussian noise)"""
    if method == 'classic':
      noise, seeds = self.sample_noise_batch(n, seeds) 
    elif method == 'mirror':
      noise, seeds = self.sample_noise_batch_mirror(n, seeds)

    new = {"kernels" : {}}
    for k in self.kernel_keys:
      new["kernels"][k] = jnp.stack([params["kernels"][k] for _ in range(n)]) \
          + noise["kernels"][k]
    for k in 'TR':
      new[k] = jnp.stack([params[k] for _ in range(n)]) + noise[k]
    new['init'] = jnp.stack([params['init'] for _ in range(n)]) + noise['init']

    return self.clip(new), seeds
  #-----------------------------------------------------------------------------
  def sample_noise_batch(self, n, seeds = None):
    if seeds is None :
      seeds = np.random.randint(0, 1000, len(self.spaces))
    else : 
      assert len(seeds) == len(self.spaces)

    noise = {"kernels" : {}}
    
    for i, k in enumerate(self.kernel_keys):
      shape = (n, self.nb_k, 3) if k in "abw" else (n, self.nb_k)
      noise["kernels"][k] = jax.random.normal(
          jax.random.PRNGKey(seeds[i]), shape
      ) * self.spaces[k]['mut_std']

    noise['init'] = jax.random.normal(jax.random.PRNGKey(seeds[-3]), (n, *self.init_shape)) * self.spaces['init']['mut_std']
    
    noise['T'] = jax.random.normal(jax.random.PRNGKey(seeds[-2]), (n,)) * self.spaces['T']['mut_std']

    noise['R'] = jax.random.normal(jax.random.PRNGKey(seeds[-1]), (n,)) * self.spaces['R']['mut_std']

    return noise, seeds
  #-----------------------------------------------------------------------------
  def sample_noise_batch_mirror(self, n, seeds = None):
    if n%2 : 
      n = n + 1
    noise, seeds = self.sample_noise_batch(int(n / 2), seeds)
    for k in "T R init".split():
      noise[k] = jnp.concatenate([noise[k], -noise[k]])
    for k in noise['kernels'].keys():
      noise['kernels'][k] = jnp.concatenate([noise['kernels'][k], - noise['kernels'][k]])
    return noise, seeds
    
  #-----------------------------------------------------------------------------
  def clip(self, params):
    return {
        'kernels' : {
            k : jnp.clip(params['kernels'][k], 
                         self.spaces[k]['low'], 
                         self.spaces[k]['high']) 
            for k in self.kernel_keys
        },
        'T' : jnp.clip(params['T'], self.spaces['T']['low'], self.spaces['T']['high']),
        'R' : jnp.clip(params['R'], self.spaces['R']['low'], self.spaces['R']['high']),
        'init' : jnp.clip(params['init'], 0., 1.)
    }
  #-----------------------------------------------------------------------------
  def clip_low(self, params):
    return {
        'kernels' : {
            k : jnp.clip(params['kernels'][k], 
                         self.spaces[k]['low'], 
                         jnp.inf) 
            for k in self.kernel_keys
        },
        'T' : jnp.clip(params['T'], self.spaces['T']['low'], jnp.inf),
        'R' : jnp.clip(params['R'], self.spaces['R']['low'], jnp.inf),
        'init' : jnp.clip(params['init'], 0., 1.)
    }
  #-----------------------------------------------------------------------------
  def make_seeds(self):
    return np.random.randint(0, 1000, len(self.spaces))

# Kernels and Growth functions

In [ ]:
def sigmoid(x):
  return 0.5 * (jnp.tanh(x / 2) + 1)

ker_f = lambda x, a, w, b : (b * jnp.exp( - (x[..., None] - a)**2 / w)).sum(-1)

In [ ]:
def compile_kernel_computer(SX, SY, nb_k):
  """return a jit compiled function taking as input lenia raw params and returning computed kernels (compiled params)"""
  mid = SX // 2
  def compute_kernels(params):
    """Compute kernels and return a dic containing fft kernels, T and R"""
    kernels = params['kernels']

    Ds = [ np.linalg.norm(np.mgrid[-mid:mid, -mid:mid], axis=0) / 
          ((params['R']+15) * kernels['r'][k]) for k in range(nb_k) ]  # (x,y,k)
    K = jnp.dstack([sigmoid(-(D-1)*10) * ker_f(D, kernels["a"][k], kernels["w"][k], kernels["b"][k]) 
                    for k, D in zip(range(nb_k), Ds)])
    nK = K / jnp.sum(K, axis=(0,1), keepdims=True)
    fK = jnp.fft.fft2(jnp.fft.fftshift(nK, axes=(0,1)), axes=(0,1))

    compiled_params = {k : kernels[k] for k in kernels.keys()}
    for k in ['R', 'T']:
      compiled_params[k] = params[k]
    compiled_params['fK'] = fK
    return compiled_params
  
  return jax.jit(compute_kernels)

In [ ]:
bell = lambda x, m, s: jnp.exp(-((x-m)/s)**2 / 2)

def growth(U, m, s):
  return bell(U, m, s)*2-1

# Lenia

In [ ]:
def build_system_original(SX, SY, nb_k, C, c0 = None, c1 = None):
  """returns step function State x Params(compiled) --> State"""

  if c0 is None or c1 is None :
    c0 = [0] * nb_k
    c1 = [[i for i in range(nb_k)]]
  
  def step(A, params):
    """
    Main step
    A : state of the system (SX, SY, C)
    params : compiled paremeters (dict) must contain T, m, s, h and fK (computed kernels fft)
    """
    #---------------------------Original Lenia------------------------------------
    fA = jnp.fft.fft2(A, axes=(0,1))  # (x,y,c)

    fAk = fA[:, :, c0]  # (x,y,k)

    U = jnp.real(jnp.fft.ifft2(params['fK'] * fAk, axes=(0,1)))  # (x,y,k)

    G = growth(U, params['m'], params['s']) * params['h']  # (x,y,k)

    H = jnp.dstack([ G[:, :, c1[c]].sum(axis=-1) for c in range(C) ])  # (x,y,c)

    return jnp.clip(A + (1 / params['T']) * H, 0., 1.)

  return jax.jit(step)

In [ ]:
#@title Example {vertical-output : true}
SX = SY = 256 # World dimensions
C = 1 # Number of channels
nb_k = 20 # Number of kernels
c0 = None ; c1 = None # kernel connections : by default all kernels are from channel 0 to channel 0 in 1-channel Lenia
rule_space = Rule_space(nb_k) 

ker_comp = compile_kernel_computer(SX, SY, nb_k) # Kernel computer : raw params --> compiled params

step_fn = build_system_original(SX, SY, nb_k, C, c0, c1)


params = rule_space.sample()
c_params = ker_comp(params)
steps = 500
A0 = jnp.zeros((SX, SY, C))
A0 = A0.at[SX//2-20:SX//2+20, SY//2-20:SY//2+20, 0].set(params['init'])
obs = rollout(step_fn, c_params, A0, steps, verbose = True)

with VideoWriter("ou.mp4", 120) as vid:
  for timestep in range(obs.shape[0]):
    rgb_im = obs[timestep, :, :, 0]
    vid.add(rgb_im)
  vid.show(width = 500, heigth = 500)

100%|██████████| 500/500 [00:02<00:00, 192.03it/s]


# Flow Lenia

In [ ]:
def build_system(SX : int, SY : int, nb_k : int, C : int, c0 : t.Iterable = None, 
                 c1 : t.Iterable = None, dt : float = .5, dd : int = 5, 
                 sigma : float = .65, n = 2, theta_A = None) -> t.Callable:
  """
  returns step function State x Params(compiled) --> State
  
  Params :
    SX, SY : world size
    nb_k : number of kernels
    C : number of channels
    c0 : list[int] of size nb_k where c0[i] is the source channel of kernel i
    c1 : list[list[int]] of size C where c1[i] is the liste of kernel indexes targetting channel i
    dt : float integrating timestep
    dd : max distance to look for when moving matter
    sigma : half size of the Brownian motion distribution
    n : scaling parameter for alpha
    theta_A : critical mass at which alpha = 1
  Return :
    callable : array(SX, SY, C) x params --> array(SX, SY, C)
  """

  if theta_A is None:
    theta_A = C

  if c0 is None or c1 is None :
    c0 = [0] * nb_k
    c1 = [[i for i in range(nb_k)]]
  
  xs = jnp.concatenate([jnp.arange(SX) for _ in range(SY)])
  ys = jnp.arange(SY).repeat(SX)

  x, y = jnp.arange(SX), jnp.arange(SY)
  X, Y = jnp.meshgrid(x, y)
  pos = jnp.dstack((Y, X)) + .5 #(SX, SY, 2)

  rollxs = []
  rollys = []
  for dx in range(-dd, dd+1):
    for dy in range(-dd, dd+1):
      rollxs.append(dx)
      rollys.append(dy)
  rollxs = jnp.array(rollxs)
  rollys = jnp.array(rollys)


  @partial(jax.vmap, in_axes = (0, 0, None, None))
  def step_flow(rollx, rolly, A, mus):
    rollA = jnp.roll(A, (rollx, rolly), axis = (0, 1))
    dpmu = jnp.absolute(pos[..., None] - jnp.roll(mus, (rollx, rolly), axis = (0, 1))) # (x, y, 2, c)
    sz = .5 - dpmu + sigma #(x, y, 2, c)
    area = jnp.prod(jnp.clip(sz, 0, min(1, 2*sigma)) , axis = 2) / (4 * sigma**2) # (x, y, c)
    nA = rollA * area
    return nA


  def step(A, params):
    """
    Main step
    A : state of the system (SX, SY, C)
    params : compiled paremeters (dict) must contain m, s, h and fK (computed kernels fft)
    """
    #---------------------------Original Lenia------------------------------------
    fA = jnp.fft.fft2(A, axes=(0,1))  # (x,y,c)

    fAk = fA[:, :, c0]  # (x,y,k)

    U = jnp.real(jnp.fft.ifft2(params['fK'] * fAk, axes=(0,1)))  # (x,y,k)

    G = growth(U, params['m'], params['s']) * params['h']  # (x,y,k)

    H = jnp.dstack([ G[:, :, c1[c]].sum(axis=-1) for c in range(C) ])  # (x,y,c)

    #-------------------------------FLOW------------------------------------------

    F = sobel(H) #(x, y, 2, c)

    C_grad = sobel(A.sum(axis = -1, keepdims = True)) #(x, y, 2, 1)
  
    alpha = jnp.clip((A[:, :, None, :]/theta_A)**n, .0, 1.)
    
    F = F * (1 - alpha) - C_grad * alpha
    
    mus = pos[..., None] + dt * F #(x, y, 2, c) : target positions (distribution centers)

    nA = step_flow(rollxs, rollys, A, mus).sum(axis = 0)
    
    return nA

  return jax.jit(step)

## Examples

In [ ]:
#@title 1-channel {vertical-output : true}
SX = SY = 256 # World dimensions
C = 1 # Number of channels
nb_k = 20 # Number of kernels
c0 = None ; c1 = None # kernel connections : by default all kernels are from channel 0 to channel 0 in 1-channel Lenia
rule_space = Rule_space(nb_k) 

ker_comp = compile_kernel_computer(SX, SY, nb_k) # Kernel computer : raw params --> compiled params

step_fn = build_system(SX, SY, nb_k, C, c0, c1, dd = 5, dt = .3, sigma = .65)

params = rule_space.sample()
c_params = ker_comp(params)
steps = 1000
A0 = jnp.zeros((SX, SY, C))
A0 = A0.at[SX//2-20:SX//2+20, SY//2-20:SY//2+20, 0].set(params['init'])
obs = rollout(step_fn, c_params, A0, steps, verbose = True)

with VideoWriter("ou.mp4", 120) as vid:
  for timestep in range(obs.shape[0]):
    rgb_im = obs[timestep, :, :, 0]
    vid.add(rgb_im)
  vid.show(width = 500, heigth = 500)

100%|██████████| 1000/1000 [00:08<00:00, 112.48it/s]


In [ ]:
#@title 2-channel {vertical-output : true}

SX = SY = 256 # World dimensions
C = 2 # Number of channels
connection_matrix = np.array([
    [5, 5],
    [5, 5]
]) # Connection matrix where M[i, j] = number of kernels from channel i to channel j
c0, c1 = conn_from_matrix(connection_matrix)
nb_k = int(connection_matrix.sum())
rule_space = Rule_space(nb_k) 

ker_comp = compile_kernel_computer(SX, SY, nb_k) # Kernel computer : raw params --> compiled params

step_fn = build_system(SX, SY, nb_k, C, c0, c1, dd = 5, dt = .2, sigma = .65)

params = rule_space.sample()
c_params = ker_comp(params)
steps = 1000
A0 = jnp.zeros((SX, SY, C))
A0 = A0.at[SX//2-20:SX//2+20, SY//2-20:SY//2+20, :].set(np.random.rand(40, 40, C))
obs = rollout(step_fn, c_params, A0, steps, verbose = True)

with VideoWriter("ou.mp4", 120) as vid:
  for timestep in range(obs.shape[0]):
    rgb_im = np.concatenate(
        [obs[timestep, :, :, 0, None], obs[timestep, :, :, 1, None], obs[timestep, :, :, 1, None]], axis = -1
    )
    vid.add(rgb_im)
  vid.show(width = 500, heigth = 500)

100%|██████████| 1000/1000 [00:03<00:00, 320.08it/s]


# Parameter embedding

In [ ]:
def build_system_p(SX, SY, nb_k, C, c0 = None, c1 = None, dt = .5, dd = 5, 
                   sigma = .65, mix : str = 'avg', T : float = 1.):
  """
  returns step function State x Params(compiled) --> State
  
  Params :
    SX, SY : world size
    nb_k : number of kernels
    C : number of channels
    c0 : list[int] of size nb_k where c0[i] is the source channel of kernel i
    c1 : list[list[int]] of size C where c1[i] is the liste of kernel indexes targetting channel i
    dt : float integrating timestep
    dd : max distance to look for when moving matter
    sigma : half size of the Brownian motion distribution
    mix : parameters mixing method (what to do when different parameters arrive in a same cell)
      "avg" makes a weighted avg of incoming parameters with respect to mass
      "softmax" weights each parameter by exp(mass / T)
      "stoch" : samples one of the incoming parameters following the softmax distribution
      "stoch_gene_wise" : samples each gene independently following softmax distribution as with "stoch" method
    T : temperature of the softmax distribution for softmax and stoch mixing rule
  Return :
    callable : array(SX, SY, C) x params --> array(SX, SY, C)
  """

  if c0 is None or c1 is None :
    c0 = [0] * nb_k
    c1 = [[i for i in range(nb_k)]]
  
  xs = jnp.concatenate([jnp.arange(SX) for _ in range(SY)])
  ys = jnp.arange(SY).repeat(SX)

  x, y = jnp.arange(SX), jnp.arange(SY)
  X, Y = jnp.meshgrid(x, y)
  pos = jnp.dstack((Y, X)) + .5 #(SX, SY, 2)

  rolls = []
  rollxs = []
  rollys = []
  for dx in range(-dd, dd+1):
    for dy in range(-dd, dd+1):
      rolls.append((dx, dy))
      rollxs.append(dx)
      rollys.append(dy)
  rollxs = jnp.array(rollxs)
  rollys = jnp.array(rollys)

  @partial(jax.vmap, in_axes = (0, 0, None, None, None))
  def step_flow(rollx, rolly, A, P, mus):
    rollA = jnp.roll(A, (rollx, rolly), axis = (0, 1))
    rollP = jnp.roll(P, (rollx, rolly), axis = (0, 1)) #(x, y, k)
    dpmu = jnp.absolute(pos[..., None] - jnp.roll(mus, (rollx, rolly), axis = (0, 1))) 
    sz = .5 - dpmu + sigma
    area = jnp.prod(jnp.clip(sz, 0, 1) , axis = 2) / (4 * sigma**2)
    nA = rollA * area
    return nA, rollP

  def step(A, P, params):
    """
    Main step
    A : state of the system (SX, SY, C)
    P : parameter map (SX, SY, nb_k) encoding h vector (kernel weighting)
    params : compiled paremeters (dict) must contain m, s, h and fK (computed kernels fft)
    """
    #---------------------------Original Lenia------------------------------------
    fA = jnp.fft.fft2(A, axes=(0,1))  # (x,y,c)

    fAk = fA[:, :, c0]  # (x,y,k)

    U = jnp.real(jnp.fft.ifft2(params['fK'] * fAk, axes=(0,1)))  # (x,y,k)

    G = growth(U, params['m'], params['s']) * P # (x,y,k)

    H = jnp.dstack([ G[:, :, c1[c]].sum(axis=-1) for c in range(C) ])  # (x,y,c)

    #-------------------------------FLOW------------------------------------------

    F = sobel(H) #(x, y, 2, c) : Flow

    C_grad = sobel(A.sum(axis = -1, keepdims = True)) #(x, y, 2, 1) : concentration gradient
  
    alpha = jnp.clip((A[:, :, None, :]/2)**2, .0, 1.)
    
    F = F * (1 - alpha) - C_grad * alpha
    
    mus = pos[..., None] + dt * F #(x, y, 2, c) : target positions (distribution centers)

    nA, nP = step_flow(rollxs, rollys, A, P, mus) #vmapped

    if mix == 'avg':
      nP = jnp.sum(nP * nA.sum(axis = -1, keepdims = True), axis = 0)  
      nA = jnp.sum(nA, axis = 0)
      nP = nP / (nA.sum(axis = -1, keepdims = True)+1e-10)
    
    elif mix == "softmax":
      expnA = jnp.exp(nA.sum(axis = -1, keepdims = True) / T) - 1
      nA = jnp.sum(nA, axis = 0)
      nP = jnp.sum(nP * expnA, axis = 0) / (expnA.sum(axis = 0)+1e-10) #avg rule

    elif mix == "stoch":
      categorical=jax.random.categorical(
          jax.random.PRNGKey(42), 
          jnp.log(nA.sum(axis = -1, keepdims = True)/T), 
          axis=0)
      mask=jax.nn.one_hot(categorical,num_classes=(2*dd+1)**2,axis=-1)
      mask=jnp.transpose(mask,(3,0,1,2)) 
      nP = jnp.sum(nP * mask, axis = 0)
      nA = jnp.sum(nA, axis = 0)
    
    elif mix == "stoch_gene_wise":
      mask = jnp.concatenate(
          [jax.nn.one_hot(jax.random.categorical(
                                                jax.random.PRNGKey(42), 
                                                jnp.log(nA.sum(axis = -1, keepdims = True) / T), 
                                                axis=0),
                          num_classes=(2*dd+1)**2,axis=-1)
          for _ in range(nb_k)], 
          axis = 2)
      mask=jnp.transpose(mask,(3,0,1,2)) # (2dd+1**2, x, y, nb_k)
      nP = jnp.sum(nP * mask, axis = 0)
      nA = jnp.sum(nA, axis = 0)

    return nA, nP

  return jax.jit(step)

In [ ]:
def rollout_p(step, params, n, s0, p0, f_mut = 250):
  SX, SY, C = s0.shape
  nb_k = p0.shape[-1]
  s = s0
  p = p0
  obs = np.zeros((n+1, *s0.shape))
  obs_p = np.zeros((n+1, *p0.shape))
  obs[0, ...] = s0 
  obs_p[0, ...] = p0
  for i in tqdm(range(n)):
    s, p = step(s, p, params)
    obs[i+1, ...] = s
    obs_p[i+1, ...] = p
    if not i % f_mut:
      x = np.random.randint(20, SX-20)
      y = np.random.randint(20, SY-20)
      p = p.at[x-10:x+10, y-10:y+10, :].add(np.ones((20, 20, nb_k)) * np.random.randn(1, 1, nb_k)*.5)

  return obs, obs_p

In [ ]:
#@title Example {vertical-output : true}

SX = SY = 256
C = 2
nb_k = 20
connection_matrix = np.array([
    [5, 5],
    [5, 5]
]) # Connection matrix where M[i, j] = number of kernels from channel i to channel j
c0, c1 = conn_from_matrix(connection_matrix)
rule_space = Rule_space(nb_k)

ker_comp = compile_kernel_computer(SX, SY, nb_k)

step_fn = build_system_p(SX, SY, nb_k, C, c0, c1, dd = 5, dt = .2, mix = 'stoch')

gc.collect()

params = rule_space.sample()
c_params = ker_comp(params)
steps = 1000

init_pts = [
            (60, 60),
            (190, 190),
            (128, 128),
            (60, 190),
            (190, 60)
] # Creatures initialization locations

avg_nk = 15 # number of active kernels of a creature in average
pk = avg_nk / nb_k

A = jnp.zeros((SX, SY, C))
P = jnp.zeros((SX, SY, nb_k))

for x, y in init_pts:
  
  A = A.at[x-20:x+20, y-20:y+20, :].set(np.random.rand(40, 40, C))
  
  h = jnp.array(np.random.rand(nb_k))
  
  P = P.at[x-20:x+20, y-20:y+20, :].set(
      h
  )

obs, obs_p = rollout_p(step_fn, c_params, steps, A, P, f_mut=100)

with VideoWriter("ou.mp4", 60) as vid:
  for timestep in range(obs.shape[0]):
    rgb_im = obs[timestep, ...].sum(axis = -1, keepdims = True) * \
            obs_p[timestep, :, :, :3]
    vid.add(rgb_im)
  vid.show(width = 500, heigth = 500)

gc.collect()

100%|██████████| 1000/1000 [00:12<00:00, 78.50it/s]


0